In [9]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import urllib.request
import requests
import os 
import json
import sys 
import re
import time
import tqdm

###### Import selenium dependencies
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys

%matplotlib inline

#### Set up chrome options for the browser

In this section, we can add keyboard arguments to the chrome browser as options. The website https://peter.sh/experiments/chromium-command-line-switches/ contains all the possible extensions to the chrome option class. 

In [10]:
## Set up chrome Options 

chrome_options = Options()  
chrome_options.add_argument("--headless") 

#### Spider Class

In [15]:
class hoaxSpider():
    
    def __init__(self):
        
        self.URL="http://www.warriorelihoax.com"
        self.content = self.get_request()
        self.soup = self.get_soup()
        self.cases = self.get_names_cases()
        
    def get_request(self):
        content = requests.get(self.URL).content
        return content
    
    def get_soup(self):
        soup = BeautifulSoup(self.content,'html.parser')
        return soup
    
    def get_names_cases(self):
        all_tables = self.soup.find_all(id = "cat")
        txt = ""
        for i in all_tables:
            txt+=i.text.strip()
        cases = []
        for i in txt.split("\n"):
            reg = re.findall(r"^[^\(]+", i)
            cases.append(reg)
        flat_list = [item for sublist in cases for item in sublist]
        newL = []
        for i in flat_list:
            newL.append(i.replace(u'\xa0', u'').replace(" ", "-"))
        return newL[1:] 
    
    def select_hoax_case(self, dropdown = '//*[@id="cat"]'):
        
        self.driver = webdriver.Chrome()
        self.driver.get("http://www.warriorelihoax.com")
        python_button = self.driver.find_element_by_xpath(dropdown + '/option[5]')
        python_button.click()

    def scrape_images(self, dropdown = '//*[@id="cat"]'):
        
        #init a chrome driver. Add headless option!
        self.driver = webdriver.Chrome()
        self.driver.get(self.URL)
        
        # option is the number of cases. This variable is embedded into python_button in order to automatically 
        # change the pages
        option = 1
        while option:
            time.sleep(2)
            
            try:
                python_button = self.driver.find_element_by_xpath(dropdown + '/option[' + str(option) + ']')
         
                #get the link for every page. This involves to call the get_names_cases. 
                new_URL = self.URL + "/category/" + self.cases[option-1] + "/"
                python_button.click()
                
                #scrape for the images!
                self.URL = new_URL
                self.content = self.get_request()
                self.soup = self.get_soup()
                
                find_images = self.soup.find_all("a", href = True) #{"id": re.compile(r'\d+')}
                find_text = self.soup.find_all("p")
                
                #create the directories
                dir_name = self.cases[option-1]
                
                if not os.path.exists("cases/" + dir_name):
                    os.makedirs("cases/" + dir_name)
                    
                    if not os.path.exists("cases/" + dir_name + "/Images"):
                        os.makedirs("cases/" + dir_name + "/Images")
                        
                    if not os.path.exists("cases/" + dir_name + "/Text"):
                        os.makedirs("cases/" + dir_name + "/Text")
                
                #download images and store to directory
                counter = 1
                for img in tqdm.tqdm(find_images):
                    try:
                        # retrieve the URLS from the images
                        urllib.request.urlretrieve(img["href"], "cases/" + dir_name + "/Images/" + "Img_" + str(counter) + ".jpg")
                        counter += 1
                    except:
                        pass
                    
                #download text and store to directory 
                s_text = ""
                for t in find_text:
                    tx = t.text.replace("\xa0", "")
                    with open("cases/" + dir_name + "/Text/text.txt", 'a') as file:
                        file.write(tx + "\r\n")                    
            except:
                break
            
            option += 1 
            print("---------- case done ----------")


In [16]:
spider = hoaxSpider()

In [17]:
case_names = spider.get_names_cases()

In [18]:
x = spider.scrape_images()